In [83]:
import sys, os
print("Current Working Directory:", os.getcwd())

script_path = os.path.abspath(__file__) if '__file__' in locals() else os.getcwd()
print("Script Path:", script_path)

parent_dir = os.path.abspath(os.path.join(script_path, '..'))
print("Parent Directory being added to sys.path:", parent_dir)

sys.path.append(parent_dir)

Current Working Directory: /Users/belhalkarimi/Desktop/Belhal/Tech/taitris/taitris-ai/src/tests
Script Path: /Users/belhalkarimi/Desktop/Belhal/Tech/taitris/taitris-ai/src/tests
Parent Directory being added to sys.path: /Users/belhalkarimi/Desktop/Belhal/Tech/taitris/taitris-ai/src


In [84]:
import asyncio
import fire

from taitriscore.memory.memory import Memory
from taitriscore.logs import logger
from taitriscore.roles import Planner
from taitriscore.config import CONFIG

import pdb

### Planner Agent

In [85]:
OBJECTIVE = """You are hired to lead an influencers marketing campaign for Taitris, a company selling innovative candles smelling like wood.
    You have access to instagram.
    Find the right influencers, reach out and try to do product seeding to bring some new clients to Taitris."""

In [86]:
role = Planner(profile="Planner", objective = OBJECTIVE)

In [88]:
tasks = ["Develop a task list for your Objective."]
completed_tasks = []

max_rounds = 2

In [7]:
r = 1
logger.info(f"Round - {r}")
for t in tasks:
    logger.info(f"- {t}")

Round - 1
- Develop a task list for your Objective.


In [8]:
# Step 1: Pull the first incomplete task
task = tasks.pop(0)
print(task)

Develop a task list for your Objective.


In [47]:
# Agent complete the task based on the context
todo = {}
todo["task"] = task
print(todo)
print(completed_tasks)

{'task': 'Develop a task list for your Objective.'}
[]


In [48]:
# tmp_todo = role.task_execution_agent(
#     OBJECTIVE, todo, completed_tasks[-5:]
# )

completed_task_bullets = "\n".join(
    [f"  - {todo['task']}: {todo['result']}" for todo in completed_tasks[-5:]]
)


In [50]:
ExecutionOutputTemplate = f"""
Perform one task based on the following objective: {OBJECTIVE}.

Take into account these previously completed tasks:
{completed_task_bullets}

Your task: {todo['task']}
Response:
"""

tmp_todo = ExecutionOutputTemplate
tmp_todo


'\nPerform one task based on the following objective: You are hired to lead an influencers marketing campaign for Taitris, a company selling innovative candles smelling like wood.\n    You have access to instagram.\n    Find the right influencers, reach out and try to do product seeding to bring some new clients to Taitris..\n\nTake into account these previously completed tasks:\n\n\nYour task: Develop a task list for your Objective.\nResponse:\n'

In [51]:
messages = [role._llm._default_system_msg(), role._llm._user_msg(tmp_todo)]
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': '\nPerform one task based on the following objective: You are hired to lead an influencers marketing campaign for Taitris, a company selling innovative candles smelling like wood.\n    You have access to instagram.\n    Find the right influencers, reach out and try to do product seeding to bring some new clients to Taitris..\n\nTake into account these previously completed tasks:\n\n\nYour task: Develop a task list for your Objective.\nResponse:\n'}]

In [52]:
role._llm._cons_kwargs(messages)

{'model': 'gpt-3.5-turbo',
 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'},
  {'role': 'user',
   'content': '\nPerform one task based on the following objective: You are hired to lead an influencers marketing campaign for Taitris, a company selling innovative candles smelling like wood.\n    You have access to instagram.\n    Find the right influencers, reach out and try to do product seeding to bring some new clients to Taitris..\n\nTake into account these previously completed tasks:\n\n\nYour task: Develop a task list for your Objective.\nResponse:\n'}],
 'max_tokens': 500,
 'n': 1,
 'stop': None,
 'temperature': 0.3}

In [64]:
tmp_input = role._llm._cons_kwargs(messages)

In [65]:
# todo["result"] = role._llm.aask(tmp_todo)
res = role._llm.llm.ChatCompletion.create(**tmp_input)

In [67]:
res['choices'][0].message.content

"Task List for Leading Influencer Marketing Campaign for Taitris:\n\n1. Research Influencers: Identify relevant influencers in the home decor, lifestyle, and wellness niches on Instagram who have a strong following and engagement rates.\n\n2. Evaluate Influencers: Assess each influencer's audience demographics, engagement levels, authenticity, and alignment with Taitris' brand values and target market.\n\n3. Reach Out to Influencers: Craft personalized outreach messages to introduce Taitris' innovative wood-scented candles and propose a collaboration for product seeding.\n\n4. Negotiate Partnerships: Discuss terms of collaboration with interested influencers, including the number of posts, stories, or other content, as well as compensation or exchange for product seeding.\n\n5. Send Product Samples: Coordinate the delivery of Taitris' candles to selected influencers, ensuring timely and safe shipping to maintain product quality.\n\n6. Monitor Influencer Content: Track influencers' post

In [70]:
todo["result"] = res
todo["result"] = todo["result"]['choices'][0].message.content

In [71]:
todo

{'task': 'Develop a task list for your Objective.',
 'result': "Task List for Leading Influencer Marketing Campaign for Taitris:\n\n1. Research Influencers: Identify relevant influencers in the home decor, lifestyle, and wellness niches on Instagram who have a strong following and engagement rates.\n\n2. Evaluate Influencers: Assess each influencer's audience demographics, engagement levels, authenticity, and alignment with Taitris' brand values and target market.\n\n3. Reach Out to Influencers: Craft personalized outreach messages to introduce Taitris' innovative wood-scented candles and propose a collaboration for product seeding.\n\n4. Negotiate Partnerships: Discuss terms of collaboration with interested influencers, including the number of posts, stories, or other content, as well as compensation or exchange for product seeding.\n\n5. Send Product Samples: Coordinate the delivery of Taitris' candles to selected influencers, ensuring timely and safe shipping to maintain product qua

In [72]:
# Step 2: Store the result in completed task
completed_tasks.append(todo)
completed_tasks

[{'task': 'Develop a task list for your Objective.',
  'result': "Task List for Leading Influencer Marketing Campaign for Taitris:\n\n1. Research Influencers: Identify relevant influencers in the home decor, lifestyle, and wellness niches on Instagram who have a strong following and engagement rates.\n\n2. Evaluate Influencers: Assess each influencer's audience demographics, engagement levels, authenticity, and alignment with Taitris' brand values and target market.\n\n3. Reach Out to Influencers: Craft personalized outreach messages to introduce Taitris' innovative wood-scented candles and propose a collaboration for product seeding.\n\n4. Negotiate Partnerships: Discuss terms of collaboration with interested influencers, including the number of posts, stories, or other content, as well as compensation or exchange for product seeding.\n\n5. Send Product Samples: Coordinate the delivery of Taitris' candles to selected influencers, ensuring timely and safe shipping to maintain product q

In [73]:
# # Step 3: Create new tasks re-prioritize task list
# newtodo = await self.task_creation_agent(
#     objective, todo["task"], todo["result"], tasks
# )

newtodo = f"""
        You are to use the result from an execution agent to create new tasks with the following objective: {OBJECTIVE}.
        The last completed task has the result:
        {todo["result"]}
        This result was based on this task description: {todo["task"]}.

        These are {len(tasks)} incomplete tasks: {', '.join(tasks)}

        Based on the result, return a list of tasks to be completed in order to meet the objective.
        These new tasks must not overlap with incomplete tasks.
        """

In [74]:
# new_task = await self._llm.aask(newtodo)
messages = [role._llm._default_system_msg(), role._llm._user_msg(newtodo)]
tmp_input = role._llm._cons_kwargs(messages)
res = role._llm.llm.ChatCompletion.create(**tmp_input)

In [76]:
new_tasks = [new_task['choices'][0].message.content]
new_tasks

["Task List for Leading Influencer Marketing Campaign for Taitris:\n\n1. Analyze Influencer Posts: Review the content and engagement of influencers' previous posts to ensure they align with Taitris' brand image and target audience.\n\n2. Create Product Seeding Strategy: Develop a detailed plan for how Taitris' wood-scented candles will be showcased by influencers, including key messaging points and unique selling propositions.\n\n3. Develop Tracking System: Set up a system to monitor the performance of each influencer collaboration, tracking metrics such as reach, engagement, and conversion rates.\n\n4. Engage with Influencers' Audience: Interact with the followers of influencers who feature Taitris' candles, responding to comments, questions, and feedback to build brand credibility.\n\n5. Optimize Influencer Partnerships: Continuously evaluate the effectiveness of influencer collaborations and adjust strategies as needed to maximize results for Taitris.\n\n6. Collaborate on Giveaways:

In [77]:
logger.info("Adding new tasks to task_storage")
for t in new_tasks:
    logger.info(f"- {t}")

Adding new tasks to task_storage
- Task List for Leading Influencer Marketing Campaign for Taitris:

1. Analyze Influencer Posts: Review the content and engagement of influencers' previous posts to ensure they align with Taitris' brand image and target audience.

2. Create Product Seeding Strategy: Develop a detailed plan for how Taitris' wood-scented candles will be showcased by influencers, including key messaging points and unique selling propositions.

3. Develop Tracking System: Set up a system to monitor the performance of each influencer collaboration, tracking metrics such as reach, engagement, and conversion rates.

4. Engage with Influencers' Audience: Interact with the followers of influencers who feature Taitris' candles, responding to comments, questions, and feedback to build brand credibility.

5. Optimize Influencer Partnerships: Continuously evaluate the effectiveness of influencer collaborations and adjust strategies as needed to maximize results for Taitris.

6. Coll

In [79]:
# Re-prioritize tasks
# finaltodo = await self.task_prioritization_agent(
#     objective, new_tasks + tasks
# )
task_bullets = "\n".join([f"  - {task}" for task in new_tasks + tasks])
finaltodo = f"""
You are tasked with prioritizing the following tasks:
{task_bullets}

Consider the ultimate objective of your team: {OBJECTIVE}.
"""

In [80]:
# tasks = await self._llm.aask(finaltodo)
messages = [role._llm._default_system_msg(), role._llm._user_msg(finaltodo)]
tmp_input = role._llm._cons_kwargs(messages)
tasks = role._llm.llm.ChatCompletion.create(**tmp_input)
tasks = tasks['choices'][0].message.content

In [81]:
logger.info(tasks)

Based on the ultimate objective of leading an influencer marketing campaign for Taitris, the tasks can be prioritized as follows:

1. **Analyzing Influencer Posts**: This task is crucial as it sets the foundation for selecting the right influencers whose content aligns with Taitris' brand image and target audience.

2. **Monitoring Competitor Influencer Campaigns**: Understanding what competitors are doing in the influencer marketing space can provide valuable insights and help in identifying opportunities for differentiation.

3. **Creating Product Seeding Strategy**: Developing a detailed plan for how Taitris' wood-scented candles will be showcased by influencers is essential for effectively showcasing the product to the target audience.

4. **Developing Tracking System**: Setting up a system to monitor the performance of influencer collaborations is important for measuring the success of the campaign and optimizing strategies.

5. **Engaging with Influencers' Audience**: Interacting

In [82]:
# return tasks
tasks = [tasks]

### Lead Generator (Influencers contact scrapper)

### OutReach Agent (GMail function/tool calling)

In [93]:
from taitriscore.config import CONFIG

In [90]:
import json 

def send_email_tool(to: str, subject:str, body:str):
    message = {
        "from": "firstname.lastname@gmail.com",
        "to": to,
        "subject": subject,
        "body": body
    }
    return json.dumps(message)

In [92]:
def draft_email(content: str):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user",
                   "content": content}],
        functions=[
            {
                "name": "send_email_tool",
                "description": "reach out to an influencer",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "to": {
                            "type": "string",
                            "description": "email address of receiver, e.g. firstname.lastname@gmail.com",
                        },
                        "subject": {
                            "type": "string",
                            "description": "subject of the email, e.g. Hi there! We have a cool product to tell you about!"
                        },
                        "body": {
                            "type": "string",
                            "description": "content of the email, e.g. Would you bhe interested in promoting our products in exchange of free products?"
                        },
                    },
                    "required": ["to", "subject", "body"],
                },
            }
        ],
        function_call="auto",
    )

    reply_content = completion.choices[0].message
    output = reply_content['function_call']['arguments']
    output = json.loads(output.replace("\n", ""))

    return output

In [94]:
from email.message import EmailMessage

def email_sender(subject, body):
    try:
        sender_email = CONFIG.GMAIL_ADDRESS #company/sender email
        sender_password = CONFIG.gmail_password #company/sender password
        receiver_email = RECEIVER_EMAIL #always change depending on the list of influencers we gathered

        msg = EmailMessage()
        msg.set_content(body)
        msg["Subject"] = subject
        msg["From"] = sender_email
        msg["To"] = receiver_email

        session = smtplib.SMTP(SMTP, PORT)
        session.starttls()
        session.login(sender_email, sender_password)
        session.send_message(msg, sender_email, receiver_email)
        session.quit()
        print("Mail Sent!")
    except Exception as e:
        print(f'Error sending email: {e}')

In [ ]:
content = "Send an email to the influencer found by the Lead Generator agent to reach out about a collaboration with the brand. Product seeding mainly."
email_content = draft_email(content=content)

In [ ]:
print(email_content)

In [ ]:
# email_sender(subject=email_content['subject'], body=email_content['body'])